In [1]:
%%capture
#Installing required Libraries wherever the code is being executed - run only if needed
!pip install soundex
!pip install jellyfish
!pip install pandasgui

In [2]:
#Importing required Libraries
import pandas as pd 
import jellyfish
import collections
import numpy as np
import soundex as sdx
#import fuzzy
#import pandasql as psql

In [13]:
# Please remember to navigate to "Shared with me" on your Gdrive home sceen
# Right click on the "HNY_Project" (the primary folder for this project) and select "Add shortcut to Drive"
# This way the below code can find a link to the Project dr folder through your own drive.

# Mounting your personal Gdrive to the Colab notebook
from google.colab import drive
drive.mount('/gdrive',force_remount=True)

Mounted at /gdrive


In [14]:
# Reading City Directory Data
BK_name = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/Pre Final/BK_1850_name.csv')
MN_name = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Standardization Output/Pre Final/MN_1850_name.csv')

# Selecting required Columns
BK_name = BK_name[['f_first_name', 'f_last_name', 'f_middle_name']]
MN_name = MN_name[['f_first_name', 'f_last_name', 'f_middle_name']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
# Renaming columns
BK_name.columns = ['first_name', 'last_name', 'middle_name']
MN_name.columns = ['first_name', 'last_name', 'middle_name']

In [16]:
# Reading Census Data
BK_census_name = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/spatial_linkage/Data/1. Input/Brooklyn/1850/census_1850_p_bk.csv')
MN_census_name = pd.read_csv('/gdrive/My Drive/HNYC_Project/Projects/spatial_linkage/Data/1. Input/Manhattan/1850/census_1850_mn_20200921.csv')

# Selecting required Columns
BK_census_name = BK_census_name[['CENSUS_FIRST_NAME', 'CENSUS_LAST_NAME']]
MN_census_name = MN_census_name[['CENSUS_FIRST_NAME', 'CENSUS_LAST_NAME']]

In [17]:
# Adding empty Middle name column
BK_census_name["middle_name"] = ""
MN_census_name["middle_name"] = ""

# Renaming columns to match CD
BK_census_name.columns = ['first_name', 'last_name', 'middle_name']
MN_census_name.columns = ['first_name', 'last_name', 'middle_name']

In [20]:
# Appending all 4 dataframes to create one final data frame
name_1850 = MN_name.append(BK_name)
name_1850 = name_1850.append(BK_census_name)
name_1850 = name_1850.append(MN_census_name)

In [21]:
# Convert Entire Dataframe to lower case
name_1850.apply(lambda x: x.astype(str).str.lower())

# Dropping Duplicate rows considering all columns
name_1850.drop_duplicates(inplace=True)

In [22]:
name_1850.describe()

,first_name,last_name,middle_name
count,486013,489873,427499
unique,33890,91411,2625
top,MARY,SMITH,
freq,15479,1948,422277


# Phonetic Encoding of Names using different algorithms

In [24]:
def soundex_encoder(x):
  if pd.notnull(x):
    x = jellyfish.soundex(x)
  else:
    pass
  return x

def metaphone_encoder(x):
  if pd.notnull(x):
    x = jellyfish.metaphone(x)
  else:
    pass
  return x

In [25]:
def sx_generator(in_df):
  out_df = pd.DataFrame()

  for index, row in in_df.iterrows():
    try:
      ret_val = row.apply(soundex_encoder)
    except Exception:
      pass
    out_df = out_df.append(ret_val,ignore_index=True)  

  out_df.columns = ['first_name_sx', 'last_name_sx', 'middle_name_sx']

  return out_df

def metaphone_generator(in_df):
  out_df = pd.DataFrame()

  for index, row in in_df.iterrows():
    try:
      ret_val = row.apply(metaphone_encoder)
    except Exception:
      pass
    out_df = out_df.append(ret_val,ignore_index=True)  

  out_df.columns = ['first_name_mph', 'last_name_mph', 'middle_name_mph']

  return out_df

In [26]:
#name_1850_enc = pd.merge(metaphone_generator(name_sample), sx_generator(name_sample), left_index=True, right_index=True)
#name_1850_enc = pd.merge(name_sample,name_1850_enc, left_index=True, right_index=True)

## Soundex Count & List Generation

In [ ]:
name_1850_sx_enc = pd.merge(name_1850, sx_generator(name_1850), left_index=True, right_index=True)

In [ ]:
name_1850_sx_enc['first_name_sx_count'] = name_1850_sx_enc.groupby('first_name_sx')['first_name_sx'].transform('count')
name_1850_sx_enc['middle_name_sx_count'] = name_1850_sx_enc.groupby('middle_name_sx')['middle_name_sx'].transform('count')
name_1850_sx_enc['last_name_sx_count'] = name_1850_sx_enc.groupby('last_name_sx')['last_name_sx'].transform('count')

In [ ]:
# List of Names for the same enconding 
working_df = name_1850_sx_enc.groupby(['first_name_sx']).agg({'first_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['first_name_sx', 'first_name_sx_list']

name_1850_sx_enc = pd.merge(name_1850_sx_enc, 
                        working_df, 
                        left_on= 'first_name_sx', 
                        right_on='first_name_sx')

# List of Names for the same enconding 
working_df = name_1850_sx_enc.groupby(['middle_name_sx']).agg({'middle_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['middle_name_sx', 'middle_name_sx_list']

name_1850_sx_enc = pd.merge(name_1850_sx_enc, 
                        working_df, 
                        left_on= 'middle_name_sx', 
                        right_on='middle_name_sx')

# List of Names for the same enconding 
working_df = name_1850_sx_enc.groupby(['last_name_sx']).agg({'last_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['last_name_sx', 'last_name_sx_list']

name_1850_sx_enc = pd.merge(name_1850_sx_enc, 
                        working_df, 
                        left_on= 'last_name_sx', 
                        right_on='last_name_sx')

## Metaphone Count & List Generation

In [ ]:
name_1850_mph_enc = pd.merge(name_1850_run, metaphone_generator(name_1850_run), left_index=True, right_index=True)

In [ ]:
name_1850_mph_enc['first_name_mph_count'] = name_1850_mph_enc.groupby('first_name_mph')['first_name_mph'].transform('count')
name_1850_mph_enc['middle_name_mph_count'] = name_1850_mph_enc.groupby('middle_name_mph')['middle_name_mph'].transform('count')
name_1850_mph_enc['last_name_mph_count'] = name_1850_mph_enc.groupby('last_name_mph')['last_name_mph'].transform('count')

In [ ]:
# List of Names for the same enconding 
working_df = name_1850_mph_enc.groupby(['first_name_mph']).agg({'first_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['first_name_mph', 'first_name_mph_list']

name_1850_mph_enc = pd.merge(name_1850_mph_enc, 
                        working_df, 
                        left_on= 'first_name_mph', 
                        right_on='first_name_mph')

# List of Names for the same enconding 
working_df = name_1850_mph_enc.groupby(['middle_name_mph']).agg({'middle_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['middle_name_mph', 'first_name_mph_list']

name_1850_mph_enc = pd.merge(name_1850_mph_enc, 
                        working_df, 
                        left_on= 'middle_name_mph', 
                        right_on='middle_name_mph')

# List of Names for the same enconding 
working_df = name_1850_mph_enc.groupby(['last_name_mph']).agg({'last_name': lambda x: set(x.str.lower())})
working_df.reset_index(level=0, inplace=True)
working_df.columns = ['last_name_mph', 'first_name_mph_list']

name_1850_mph_enc = pd.merge(name_1850_mph_enc, 
                        working_df, 
                        left_on= 'last_name_mph', 
                        right_on='last_name_mph')

# JSON Generation 

In [ ]:
sample = name_1850_sx_enc.groupby(['last_name_sx']).agg({'last_name': lambda x: set(x.str.lower())})
sample

In [ ]:
print(sample.set_index('last_name_sx').rename(columns={'last_name':'index1'}).to_json())

In [ ]:
list(sample.iloc[1,8])

['jane', 'johanna', 'joanna', 'jean', 'jona', 'jno', 'john', 'johann', 'jahne']

# Exploration

In [ ]:
name_1850_sx_enc.sort_values(by='first_name_sx_count', ascending=False).head(40)

# Writing Data

In [ ]:
# Writing the dataframe to the final output folder
name_1850_sx_enc.to_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Name Dictionaries/name_1850_sx_enc ', index=False)
name_1850_mph_enc.to_csv('/gdrive/My Drive/HNYC_Project/Projects/standardization/Data/Name Dictionaries/name_1850_mph_enc', index=False)

## Extras

In [ ]:
def df_soundex(df,in_col, out_col):
  hash_table = collections.defaultdict()
  for index, row in df.iterrows():
    try:
      if pd.isna(row[in_col]):
        hash_table[index] = float("NaN")
      else:
        hash_table[index] = jellyfish.soundex(row[in_col])
  
      df[out_col] = pd.Series(dict(hash_table),index=df.index)
    except Exception:
      pass
  return df

def df_metaphone(df,in_col, out_col):
  hash_table = collections.defaultdict()
  for index, row in df.iterrows():
      if pd.isna(row[in_col]):
        hash_table[index] = float("NaN")
      else:
        hash_table[index] = jellyfish.metaphone(row[in_col])
      
      df[out_col] = pd.Series(dict(hash_table),index=df.index)
  
  return df